In [2]:
import pandas as pd
import json 
import gzip
import os
import re as regex
import progressbar
import pickle
import numpy as np

In [29]:
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking')
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'val_cleaned')) if file.endswith('.json.gz')]
turl_input_path = os.path.join(product_path, 'TURL/input')

In [18]:
# # # use clean tables to get table representation for TURL rewritten without progressbar
# train_representation=[]
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_json(os.path.join(train_test_all_filtered_path,'test_cleaned') + '/{}'.format(zip_file), compression='gzip', lines=True)
#     if ('description' in df.columns)== False: # check if description column given
#         df['description'] = ''
#     if ('name' in df.columns)== False: # check if description column given
#         df['name'] = df.tokens
#     df_cleaned=df[df['cluster_id']!=-100].reset_index().drop(columns=['index'])
#     table_representation = [] #empty list for table
#     table_representation.append(zip_file)#append table id
#     table_representation.append('')#append page title -> not relevant
#     table_representation.append('')#append wikipedia page id -> not given
#     table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#     table_representation.append('')#append table caption -> not given
#     table_representation.append(['name','description']) #append headers -> not sure if we should do that
#     all_rows_representation = []#representation of all rows
#     column_1_representation=[] # cell representation of column 1    
#     column_2_representation=[] # cell representation of column 1
#     for i in range(len(df_cleaned)):
#         column_1_representation.append([[i,0],[df_cleaned['row_id'][i],str(df_cleaned['name'][i])]])
#         column_2_representation.append([[i,1],[df_cleaned['row_id'][i],str(df_cleaned['description'][i])]])
#     all_rows_representation.append(column_1_representation)
#     all_rows_representation.append(column_2_representation) #append single column representation to representation of all rows
#     table_representation.append(all_rows_representation)#append it to representation of whole table
#     table_representation.append(df_cleaned['cluster_id'].apply(lambda x: [str(x)]).to_list())
#     train_representation.append(table_representation)

/Product_ahorroimprimiendo.com_September2020.json.gz
/Product_watch4usale.com_September2020.json.gz
/Product_novmak.com_September2020.json.gz
/Product_camtecphoto.com_September2020.json.gz
/Product_reveresecurity.com_September2020.json.gz
/Product_comboink.com_September2020.json.gz
/Product_roysbikes.com_September2020.json.gz
/Product_weaverscarpet.com_September2020.json.gz
/Product_louisianarunning.com_September2020.json.gz
/Product_heathflooring.com_September2020.json.gz
/Product_yudinsappliances.com_September2020.json.gz
/Product_velopro.com_September2020.json.gz
/Product_billmans.com_September2020.json.gz
/Product_frenshmo.com.au_September2020.json.gz
/Product_beandcare.com_September2020.json.gz
/Product_welovetec.com_September2020.json.gz
/Product_idefix.com_September2020.json.gz
/Product_bikebarnaz.com_September2020.json.gz
/Product_bobbaileys.com_September2020.json.gz
/Product_farmabimbi.com_September2020.json.gz
/Product_dealsaday.com_September2020.json.gz
/Product_hollisterhom

/Product_codigo03.com_September2020.json.gz
/Product_camerashopmuskegon.com_September2020.json.gz
/Product_cgbsolutions.com.au_September2020.json.gz
/Product_lafbike.com_September2020.json.gz
/Product_cycleogicalbikes.com_September2020.json.gz
/Product_harveynorman.com.au_September2020.json.gz
/Product_nashbar.com_September2020.json.gz
/Product_industrialcaflor.com_September2020.json.gz
/Product_russellsfitness.com_September2020.json.gz
/Product_kixpress.com_September2020.json.gz
/Product_personalsafetysource.com_September2020.json.gz
/Product_sacramentohyundai.com_September2020.json.gz
/Product_alaada.com_September2020.json.gz
/Product_camcor.com_September2020.json.gz
/Product_screekoutfitters.com_September2020.json.gz
/Product_warehousecpt.com_September2020.json.gz
/Product_senzigs.com_September2020.json.gz
/Product_pinkdolphinsecurity.com_September2020.json.gz
/Product_lnt.com.lb_September2020.json.gz
/Product_carpetsplusdesign.com_September2020.json.gz
/Product_buycheapcentral.com_

/Product_raysappliancestaunton.com_September2020.json.gz
/Product_infomaxparis.com_September2020.json.gz
/Product_gorigogo.com_September2020.json.gz
/Product_studiodepot.com_September2020.json.gz
/Product_iphoneshoppingonline.com_September2020.json.gz
/Product_billsbikeandrun.com_September2020.json.gz
/Product_greggscycles.com_September2020.json.gz
/Product_flashmemo.co.uk_September2020.json.gz
/Product_woodcockcycle.com_September2020.json.gz
/Product_bt.com_September2020.json.gz
/Product_bonzadigital.co.uk_September2020.json.gz
/Product_kool-heads.com_September2020.json.gz
/Product_nmbikensport.com_September2020.json.gz
/Product_asus.com_September2020.json.gz
/Product_coolmobile.net.nz_September2020.json.gz
/Product_rydjor.com_September2020.json.gz
/Product_rapidfurnitureutica.com_September2020.json.gz
/Product_bandlbicycles.com_September2020.json.gz
/Product_itosolutions.net_September2020.json.gz
/Product_blueridgecyclery.com_September2020.json.gz
/Product_afterhours.co.uk_September2

/Product_gibsonnet.com.br_September2020.json.gz
/Product_winedelight.com_September2020.json.gz
/Product_donsbikeshop.com_September2020.json.gz
/Product_danibikes.com_September2020.json.gz
/Product_botach.com_September2020.json.gz
/Product_irtmn.com_September2020.json.gz
/Product_motionmakers.com_September2020.json.gz
/Product_intercompras.com_September2020.json.gz
/Product_lauderdalecyclery.com_September2020.json.gz
/Product_christianflooring.com_September2020.json.gz
/Product_sepetciniz.com_September2020.json.gz
/Product_franksappliance.com_September2020.json.gz
/Product_printersupermarket.com.au_September2020.json.gz
/Product_brightretail.co.uk_September2020.json.gz
/Product_smithtownappliances.com_September2020.json.gz
/Product_dicksmith.com.au_September2020.json.gz
/Product_mediability.com_September2020.json.gz
/Product_davesappliancewin.com_September2020.json.gz
/Product_redrivercyclery.com_September2020.json.gz
/Product_aplikasionlineshop.com_September2020.json.gz
/Product_webjet

/Product_macstore.com.pa_September2020.json.gz
/Product_kalex.com.br_September2020.json.gz
/Product_shrevecrumpandlow.com_September2020.json.gz
/Product_johnnybrinks.com_September2020.json.gz
/Product_mmf-pro.com_September2020.json.gz
/Product_donaldsonflooring.com_September2020.json.gz
/Product_shopwaltons.com_September2020.json.gz
/Product_dzustore.com_September2020.json.gz
/Product_telefonchik.net_September2020.json.gz
/Product_wcslmall.com_September2020.json.gz
/Product_hawleysbicycleworld.com_September2020.json.gz
/Product_yohohongkong.com_September2020.json.gz
/Product_rbrooksandson.com_September2020.json.gz
/Product_verpacking.com_September2020.json.gz
/Product_ebikestore.com_September2020.json.gz
/Product_joyeriaamores.com_September2020.json.gz
/Product_coloradomultisport.com_September2020.json.gz
/Product_sportshowroom.com.br_September2020.json.gz
/Product_bandtelectricinc.com_September2020.json.gz
/Product_wallerboot.com_September2020.json.gz
/Product_katzfloor.com_September2

/Product_sportwatchestopshops.com_September2020.json.gz
/Product_buyskincare.co.uk_September2020.json.gz
/Product_rez01.com.au_September2020.json.gz
/Product_deltatronicinformatica.com.br_September2020.json.gz
/Product_lovojo.com_September2020.json.gz
/Product_nasons.net_September2020.json.gz
/Product_americanpaininstitute.org_September2020.json.gz
/Product_bobscarpetmart.com_September2020.json.gz
/Product_serversdirect.co.uk_September2020.json.gz
/Product_ladolcevelo.com_September2020.json.gz
/Product_fenixcycling.com_September2020.json.gz
/Product_lloydspharmacy.com_September2020.json.gz
/Product_nzgameshop.com_September2020.json.gz
/Product_applianceworld.com_September2020.json.gz
/Product_perpetualkid.com_September2020.json.gz
/Product_solehello.com_September2020.json.gz
/Product_trekbikesofmountpleasant.com_September2020.json.gz
/Product_aussied.com_September2020.json.gz
/Product_smartbikeparts.com_September2020.json.gz
/Product_toastwines.com_September2020.json.gz
/Product_safefa

/Product_ncs.com.my_September2020.json.gz
/Product_camerapro.com.au_September2020.json.gz
/Product_365rider.com_September2020.json.gz
/Product_cactusmusictx.com_September2020.json.gz
/Product_pedalpowerbikeshop.com_September2020.json.gz
/Product_mytrendyphone.co.uk_September2020.json.gz
/Product_surecando.com.au_September2020.json.gz
/Product_telemoveis-livres.com_September2020.json.gz
/Product_bilendenal.com_September2020.json.gz
/Product_visiofactory.com_September2020.json.gz
/Product_mundotronik.com_September2020.json.gz
/Product_goddessarmorprotection.net_September2020.json.gz
/Product_toplinesuperstore.net_September2020.json.gz
/Product_ishop-mall.com_September2020.json.gz
/Product_babylonhonda.com_September2020.json.gz
/Product_prophotosupply.com_September2020.json.gz
/Product_hugosmelroseappliance.com_September2020.json.gz
/Product_designerhomeappliances.com_September2020.json.gz
/Product_vinawatch.com_September2020.json.gz
/Product_fiberoptics.com_September2020.json.gz
/Product

/Product_ontariocarpetstore.com_September2020.json.gz
/Product_evanselectrical-online.co.uk_September2020.json.gz
/Product_poshnpose.com_September2020.json.gz
/Product_giveandtakeuk.com_September2020.json.gz
/Product_9wbikeshop.com_September2020.json.gz
/Product_webmercs.com_September2020.json.gz
/Product_daltonwholesalefloors.com_September2020.json.gz
/Product_grelly.com_September2020.json.gz
/Product_bigskybike.com_September2020.json.gz
/Product_offeroftheday.co.uk_September2020.json.gz
/Product_radsecurityconcepts.com_September2020.json.gz
/Product_hoerschhomeappliance.com_September2020.json.gz
/Product_shopdownwindsports.com_September2020.json.gz
/Product_shopidc.com_September2020.json.gz
/Product_desrs.com_September2020.json.gz
/Product_bigshopper.com_September2020.json.gz
/Product_ejuicesteals.com_September2020.json.gz
/Product_acewashersupplies.com_September2020.json.gz
/Product_bhavyaji.com_September2020.json.gz
/Product_scottsbikes.com_September2020.json.gz
/Product_sebabazar.

In [ ]:
# # use clean tables to get table representation for TURL rewritten
# count = 0
# train_representation=[] #representation of whole training set
# with progressbar.ProgressBar(max_value=len(files_representation_train)) as bar:
#     for zip_file in files_representation_train:
#         print('/{}'.format(zip_file))
#         df = pd.read_json(os.path.join(train_test_all_filtered_path,'train') + '/{}'.format(zip_file), compression='gzip', lines=True)
#         if ('description' in df.columns)== False: # check if description column given
#             df['description'] = ''
#         if ('name' in df.columns)== False: # check if description column given
#             df['name'] = df.tokens
#         table_representation = [] #empty list for table
#         table_representation.append(zip_file)#append table id
#         table_representation.append('')#append page title -> not relevant
#         table_representation.append('')#append wikipedia page id -> not given
#         table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#         table_representation.append('')#append table caption -> not given
#         table_representation.append(['name','description']) #append headers -> not sure if we should do that
#         all_rows_representation = []#representation of all rows
#         for i in range(len(df)):
#             column_1_representation=[] # cell representation of column 1
#             column_2_representation=[] # cell representation of column 1
#             column_1_representation.append([[i,0],[df['row_id'][i],str(df['name'][i])]])
#             column_2_representation.append([[i,1],[df['row_id'][i],str(df['description'][i])]])
#         all_rows_representation.append(column_1_representation).append(column_2_representation) #append single column representation to representation of all rows
#         table_representation.append(all_rows_representation)#append it to representation of whole table
#         table_representation.append(df['cluster_id'].apply(lambda x: [str(x)]).to_list())
#         train_representation.append(table_representation)
#         count += 1
#         bar.update(count)

In [30]:
# # use clean tables to get table representation for TURL with transposed matrix
# train_representation=[] #representation of whole training set
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_json(os.path.join(train_test_all_filtered_path,'val_cleaned') + '/{}'.format(zip_file), compression='gzip', lines=True)
#     if ('description' in df.columns)== False: # check if description column given
#         df['description'] = ''
#     if ('name' in df.columns)== False: # check if description column given
#         df['name'] = df.tokens
#     df['header']='Product'#empty string for headers
#     df_cleaned=df[df['cluster_id']!=-100].reset_index().drop(columns=['index'])#get rid of clusters with -100 as data is too much
#     table_representation = [] #empty list for table
#     table_representation.append(zip_file)#append table id
#     table_representation.append('')#append page title -> not relevant
#     table_representation.append('')#append wikipedia page id -> not given
#     table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#     table_representation.append('')#append table caption -> not given
#     table_representation.append(df_cleaned['header'].to_list()) #append headers -> not sure if we should do that
#     all_rows_representation = []#representation of all rows
#     for i in range(len(df_cleaned)):
#         row_representation=[] # cell representation of single row
#         row_representation.append([[0,i],[df_cleaned['row_id'][i],str(df_cleaned['name'][i])]])
#         row_representation.append([[1,i],[df_cleaned['row_id'][i],str(df_cleaned['description'][i])]])
#         all_rows_representation.append(row_representation) #append single cell representation to representation of all rows
#     table_representation.append(all_rows_representation)#append it to representation of whole table
#     table_representation.append(df_cleaned['cluster_id'].apply(lambda x: [str(x)]).to_list())
#     train_representation.append(table_representation)

/Product_bspbikes.com_September2020.json.gz
/Product_northcentralcyclery.com_September2020.json.gz
/Product_grandrivercycle.com_September2020.json.gz
/Product_cyclesgalleria.com.au_September2020.json.gz
/Product_expotronica.com_September2020.json.gz
/Product_carpetoutletpelham.com_September2020.json.gz
/Product_carolinatriathlon.com_September2020.json.gz
/Product_sprocketandgear.co.uk_September2020.json.gz
/Product_artemisoverland.com_September2020.json.gz
/Product_mervisdiamond.com_September2020.json.gz
/Product_sheelys.com_September2020.json.gz
/Product_cartridges-directuk.com_September2020.json.gz
/Product_ubuy.com.kw_September2020.json.gz
/Product_salaostore.com.br_September2020.json.gz
/Product_zeuscomics.com_September2020.json.gz
/Product_grandoptical.com_September2020.json.gz
/Product_laptopsupermarket.com.au_September2020.json.gz
/Product_lablaab.com_September2020.json.gz
/Product_deanimeydemanga.com_September2020.json.gz
/Product_piipoo.com_September2020.json.gz
/Product_troys

/Product_movatik.com_September2020.json.gz
/Product_premort.com.ng_September2020.json.gz
/Product_calistogabikeshop.com_September2020.json.gz
/Product_mainstreetflooringanddesign.com_September2020.json.gz
/Product_genesisbicycles.com_September2020.json.gz
/Product_lapinourose.com_September2020.json.gz
/Product_eobuwie.com.pl_September2020.json.gz
/Product_pharmaholic.com_September2020.json.gz
/Product_centervillewesternstores.com_September2020.json.gz
/Product_sizeer.com_September2020.json.gz
/Product_mykonfig.com_September2020.json.gz
/Product_tintasbaratas.com_September2020.json.gz
/Product_landamex.com_September2020.json.gz
/Product_tiendasarcadia.com_September2020.json.gz
/Product_buylocalnow.com_September2020.json.gz
/Product_o-video.co.uk_September2020.json.gz
/Product_beebeedesigns.com_September2020.json.gz
/Product_dailysteals.com_September2020.json.gz
/Product_kindinformatique.com_September2020.json.gz
/Product_dafasystem.com_September2020.json.gz
/Product_beatcycles.com_Septe

/Product_xxldeals.com_September2020.json.gz
/Product_folsombike.com_September2020.json.gz
/Product_ilfarmacoacasa.com_September2020.json.gz
/Product_designertagged.com_September2020.json.gz
/Product_shieldofsafety.com_September2020.json.gz
/Product_mbrochester.com_September2020.json.gz
/Product_alltricks.com_September2020.json.gz
/Product_machineryrowbicycles.com_September2020.json.gz
/Product_rosendaleflooring.com_September2020.json.gz
/Product_sockdrawer.com_September2020.json.gz
/Product_thehubwestlake.com_September2020.json.gz
/Product_abcbikes.com.au_September2020.json.gz
/Product_laptopfirsati.com_September2020.json.gz
/Product_pinnaclehostingservice.com_September2020.json.gz
/Product_animauxbouffe.com_September2020.json.gz
/Product_kharidiye.com_September2020.json.gz
/Product_dermadea.com_September2020.json.gz
/Product_rinersinc.com_September2020.json.gz
/Product_chtralee.com_September2020.json.gz
/Product_interwatches.com_September2020.json.gz
/Product_xn--72ca3bzdvbbcb1eb5icp.

/Product_grandcentralhomefurnishings.com_September2020.json.gz
/Product_arlingtonvelosport.com_September2020.json.gz
/Product_maplegrovecycling.com_September2020.json.gz
/Product_wagnerrug.com_September2020.json.gz
/Product_chrono24.com_September2020.json.gz
/Product_landrysinc.com_September2020.json.gz
/Product_onlineshopthodupuzha.com_September2020.json.gz
/Product_focusbikes.com.co_September2020.json.gz
/Product_smmcyclery.com_September2020.json.gz
/Product_element14.com_September2020.json.gz
/Product_aircraftspruce.com_September2020.json.gz
/Product_jaxbicycles.com_September2020.json.gz
/Product_worldoffloorsflorida.com_September2020.json.gz
/Product_biketechcf.com_September2020.json.gz
/Product_macwizardservice.com_September2020.json.gz
/Product_iye.com_September2020.json.gz
/Product_harleysbicycles.com_September2020.json.gz
/Product_drogariavenancio.com.br_September2020.json.gz
/Product_iamvelo.com_September2020.json.gz
/Product_steffans.co.uk_September2020.json.gz
/Product_parra

/Product_grisepet.com_September2020.json.gz
/Product_winnershopbd.com_September2020.json.gz
/Product_minipriceexpress.com_September2020.json.gz
/Product_kongcomputers.com.au_September2020.json.gz
/Product_drugstoresandomenico.com_September2020.json.gz
/Product_floorinspirations.com_September2020.json.gz
/Product_1fodiscount.com_September2020.json.gz
/Product_marzucoelectric.com_September2020.json.gz
/Product_demirbilisim.net_September2020.json.gz
/Product_smoothsales.com.au_September2020.json.gz
/Product_inventus.com.tr_September2020.json.gz
/Product_treasuresuc.com_September2020.json.gz
/Product_ilovecomputers.com.au_September2020.json.gz
/Product_slimandknobbys.com_September2020.json.gz
/Product_alexbicycles.com_September2020.json.gz
/Product_relojesdemoda.com_September2020.json.gz
/Product_ebikeofcolorado.com_September2020.json.gz
/Product_cress-sport.com_September2020.json.gz
/Product_webbfloors.com_September2020.json.gz
/Product_bikedr.com_September2020.json.gz
/Product_ncrangola.

/Product_ghsell.com_September2020.json.gz
/Product_pccasegear.com_September2020.json.gz
/Product_jbl.com.ph_September2020.json.gz
/Product_giantrenosparks.com_September2020.json.gz
/Product_bikemiamibeach.com_September2020.json.gz
/Product_headieshideout.com_September2020.json.gz
/Product_pharmacyclub.net_September2020.json.gz
/Product_contebikes.com_September2020.json.gz
/Product_jeffsappliance.com_September2020.json.gz
/Product_informaticalacronica.com_September2020.json.gz
/Product_wetri.com.au_September2020.json.gz
/Product_simplecellshop.com_September2020.json.gz
/Product_joieriagrau.com_September2020.json.gz
/Product_newlentes.com.br_September2020.json.gz
/Product_theflooringcenterwv.com_September2020.json.gz
/Product_cameraworld.co.uk_September2020.json.gz
/Product_watchobsession.co.uk_September2020.json.gz
/Product_superiorsoundnwt.com_September2020.json.gz
/Product_thegpsstore.com_September2020.json.gz
/Product_ugurfoto.com_September2020.json.gz
/Product_nutritioncentralusa.co

In [30]:
# with open(os.path.join(turl_input_path, 'train_representation_header_product.txt'), "wb") as fp:  #Pickling
#     pickle.dump(train_representation, fp)

In [31]:
# files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'val')) if file.endswith('.json.gz')]
# with open(os.path.join(turl_input_path, 'val_representation_header_product.txt'), "rb") as fp:   # Unpickling
#     train_representation = pickle.load(fp)
# with open(os.path.join(turl_input_path, 'val_representation_header_product_small.txt'), "wb") as fp:  #Pickling
#     pickle.dump(train_representation[:10], fp)

In [6]:
# # generate type_vocab.txt
# df_train= pd.read_json(os.path.join(train_test_all_filtered_path,'train/concatenated_data/train_all_filtered_tables.json.gz'), compression='gzip', lines=True)
# df_test= pd.read_json(os.path.join(train_test_all_filtered_path,'test/concatenated_data/test_all_filtered_tables.json.gz'), compression='gzip', lines=True)
# df_val= pd.read_json(os.path.join(train_test_all_filtered_path,'val/concatenated_data/val_all_filtered_tables.json.gz'), compression='gzip', lines=True)

In [15]:
# ids = df_train.cluster_id.astype(str).unique()
# ids = np.append(ids,'-100')
# len(ids)

1411

In [16]:
#save list of cluster ids to txt file
#pd.DataFrame(ids).to_csv(path_or_buf=os.path.join(turl_input_path, 'type_vocab_clusters.txt'),sep='\t', index=True, header=False)

In [8]:
# #try to save data as json
# files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'train')) if file.endswith('.json.gz')]
# with open(os.path.join(turl_input_path, 'train_representation_header_product.txt'), "rb") as fp:   # Unpickling
#     train_representation = pickle.load(fp)

In [31]:
# len(train_representation)

885

In [11]:
# #cannot read numpy integers in json -> encoder for saving
# import json
# import numpy as np

# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super(NpEncoder, self).default(obj)

In [32]:
# with open(os.path.join(turl_input_path, 'val_cleaned_representation_header_product_cleaned.json'), 'w') as f:
#     json.dump(json.dumps(train_representation, cls=NpEncoder), f)

# Representation for LocalBusiness

In [68]:
product_path = '../../../../src/data/LocalBusiness'
train_test_all_filtered_path = os.path.join(product_path, 'Splitting_ManualCheck/Train_Validation_Test')
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'validation_tables_cleaned')) if file.endswith('.csv')]
turl_input_path = os.path.join(product_path, 'TURL/input')

In [69]:
# # use clean tables to get table representation for TURL rewritten without progressbar
train_representation=[]
for zip_file in files_representation_train:
    print('/{}'.format(zip_file))
    df = pd.read_csv(os.path.join(train_test_all_filtered_path,'validation_tables_cleaned') + '/{}'.format(zip_file))
    if ('name' in df.columns)== False: # check if description column given
        df['name'] = df.page_url
    if df[df['cluster_id']!=-100].empty == False: #check if table is empty
        df_cleaned=df[df['cluster_id']!=-100].reset_index().drop(columns=['index'])
        table_representation = [] #empty list for table
        table_representation.append(zip_file)#append table id
        table_representation.append('')#append page title -> not relevant
        table_representation.append('')#append wikipedia page id -> not given
        table_representation.append('')#append information about entity -> product, not relevant since same for all tables
        table_representation.append('')#append table caption -> not given
        table_representation.append(['name']) #append headers -> not sure if we should do that
        all_rows_representation = []#representation of all rows
        column_1_representation=[] # cell representation of column 1    
        for i in range(len(df_cleaned)):
            column_1_representation.append([[i,0],[df_cleaned.index[i],str(df_cleaned['name'][i])]])
        all_rows_representation.append(column_1_representation)#append single column representation to representation of all rows
        table_representation.append(all_rows_representation)#append it to representation of whole table
        table_representation.append(df_cleaned['cluster_id'].apply(lambda x: [str(x)]).to_list())
        train_representation.append(table_representation)

/Restaurant_sluurpy.com_September2020.csv
/LocalBusiness_hautemessblog.com_September2020.csv
/LocalBusiness_danishfjordholiday.com_September2020.csv
/LocalBusiness_tweedehands-auto-kopen-en-verkopen.nl_September2020.csv
/LocalBusiness_prolawguide.com_September2020.csv
/Hotel_elenahotelmikonos.com_September2020.csv
/LocalBusiness_visitdelaware.com_September2020.csv
/LocalBusiness_06252.com.ua_September2020.csv
/LocalBusiness_bursamport.com_September2020.csv
/LocalBusiness_chamberdata.cc_September2020.csv
/LocalBusiness_agualatinoamerica.com_September2020.csv
/Hotel_5thcornerhotelstpetersburg.com_September2020.csv
/Hotel_danishfjordholiday.com_September2020.csv
/LocalBusiness_sup-training.com_September2020.csv
/LocalBusiness_lgmdeklaw.com_September2020.csv
/LocalBusiness_my-langeland.com_September2020.csv
/LocalBusiness_duboispachamber.com_September2020.csv
/LocalBusiness_venetianbags.com_September2020.csv
/Hotel_dayuse.co.uk_September2020.csv
/LocalBusiness_welkome.co_September2020.csv


/LocalBusiness_thevoice.ninja_September2020.csv
/LocalBusiness_expertschoice.net_September2020.csv
/Hotel_hotellagunazagreb.com_September2020.csv
/LocalBusiness_boarding.com_September2020.csv
/LocalBusiness_businessresource.co_September2020.csv
/LocalBusiness_visitribeesbjerg.es_September2020.csv
/LocalBusiness_best-debt-consolidation-reviews.com_September2020.csv
/LocalBusiness_visitdenmark.nl_September2020.csv
/LocalBusiness_whassupdirectory.com_September2020.csv
/LocalBusiness_kleencutirrigation.com_September2020.csv
/Hotel_halalbooking.com_September2020.csv
/LocalBusiness_visitmors.com_September2020.csv
/Hotel_hotelvanbellebrussels.com_September2020.csv
/LocalBusiness_mooresplumbingmesquite.com_September2020.csv
/LocalBusiness_asj-net.com_September2020.csv
/LocalBusiness_homify.co.th_September2020.csv
/LocalBusiness_themorningbulletin.com.au_September2020.csv
/LocalBusiness_modernwar.pl_September2020.csv
/LocalBusiness_gaycities.com_September2020.csv
/LocalBusiness_financemyboat.co

/LocalBusiness_pennsylvaniacin.com_September2020.csv
/LocalBusiness_webiraa.com_September2020.csv
/Hotel_casino-urlaub.at_September2020.csv
/LocalBusiness_dsmwestside.org_September2020.csv
/LocalBusiness_mobile4biznes.pl_September2020.csv
/LocalBusiness_visitsonderjylland.nl_September2020.csv
/Hotel_visitaarhusregion.de_September2020.csv
/Hotel_bookingsouthtyrol.com_September2020.csv
/Restaurant_chateauxhotels.com_September2020.csv
/Restaurant_sydneybars.com.au_September2020.csv
/LocalBusiness_maplemountainrecovery.com_September2020.csv
/LocalBusiness_homegrowncannabisco.com_September2020.csv
/Restaurant_melbournenightclubs.com.au_September2020.csv
/LocalBusiness_soirvine.com_September2020.csv
/LocalBusiness_phoenixnewtimes.com_September2020.csv
/LocalBusiness_midwest-contractor.com_September2020.csv
/Hotel_h2platinumhotelbelohorizonte.com_September2020.csv
/LocalBusiness_chicagofoundationpros.com_September2020.csv
/Hotel_visitnordjylland.de_September2020.csv
/Restaurant_happytrips.com

/LocalBusiness_visitlolland-falster.com_September2020.csv
/Restaurant_dinnerbooking.com_September2020.csv
/Restaurant_simon.com_September2020.csv
/LocalBusiness_may.am_September2020.csv
/Restaurant_veggieoption.com_September2020.csv
/LocalBusiness_newburyportchamber.org_September2020.csv
/Hotel_cambridgehotelontario.com_September2020.csv
/LocalBusiness_ukontheweb.co.uk_September2020.csv
/LocalBusiness_hoostels.com_September2020.csv
/Hotel_mein-langeland.de_September2020.csv
/Hotel_sensehotel-seminyak.com_September2020.csv
/LocalBusiness_banggiain.com_September2020.csv
/Hotel_francuskihotelkrakow.com_September2020.csv
/LocalBusiness_saltconstructionllc.build_September2020.csv
/Hotel_miamiandbeaches.world_September2020.csv
/LocalBusiness_weisecriminaldefense.com_September2020.csv
/LocalBusiness_localcarpetcleanersnearme.com_September2020.csv
/Hotel_oetztal.com_September2020.csv
/LocalBusiness_dirfly.net_September2020.csv
/LocalBusiness_dominickfiorenzacpa.com_September2020.csv
/LocalBusi

/Restaurant_major-foodie.com_September2020.csv
/LocalBusiness_globuya.com_September2020.csv
/LocalBusiness_liposuctionofnyc.com_September2020.csv
/LocalBusiness_blsj.com_September2020.csv
/Hotel_hotelmasainternationaltorrevieja.com_September2020.csv
/LocalBusiness_classyweb.biz_September2020.csv
/LocalBusiness_expertise.com_September2020.csv
/LocalBusiness_nephrocare.com_September2020.csv
/LocalBusiness_usharbors.com_September2020.csv
/Restaurant_michelin.com_September2020.csv
/LocalBusiness_indianainpatientrehab.com_September2020.csv
/LocalBusiness_webshutl.com_September2020.csv
/LocalBusiness_financebrokersnearme.com.au_September2020.csv
/LocalBusiness_staffnearme.com.au_September2020.csv
/LocalBusiness_visittenerife.ru_September2020.csv
/Restaurant_foodnetwork.com_September2020.csv
/LocalBusiness_visitthy.com_September2020.csv
/LocalBusiness_05447.com.ua_September2020.csv
/LocalBusiness_06178.com.ua_September2020.csv
/LocalBusiness_0619.com.ua_September2020.csv
/Hotel_hoteldabolsapo

/Restaurant_islandsindenmark.com_September2020.csv
/LocalBusiness_kiddyjewel.eu_September2020.csv
/Hotel_kayak.com_September2020.csv
/Hotel_marianovaloungehoteltavira.com_September2020.csv
/Hotel_alrahabeachhotel-abudhabi.com_September2020.csv
/Hotel_visitdallas.com_September2020.csv
/LocalBusiness_visitcottonwoodaz.org_September2020.csv
/LocalBusiness_vickremodeling.com_September2020.csv
/Restaurant_restaurant-flammen.dk_September2020.csv
/LocalBusiness_gogodex.com_September2020.csv
/LocalBusiness_officialdirectory.co_September2020.csv
/LocalBusiness_webtenerife.de_September2020.csv
/LocalBusiness_pearlsoftheweb.org_September2020.csv
/LocalBusiness_otlusa.biz_September2020.csv
/Restaurant_com.ru_September2020.csv


In [55]:
# # use clean tables to get table representation for TURL with transposed matrix
# train_representation=[] #representation of whole training set
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_csv(os.path.join(train_test_all_filtered_path,'train_tables_cleaned') + '/{}'.format(zip_file))
#     if ('name' in df.columns)== False: # check if description column given
#         df['name'] = df.page_url
#     df['header']='LocalBusiness'#empty string for headers
#     if df[df['cluster_id']!=-100].empty == False: #check if table is empty
#         df_cleaned=df[df['cluster_id']!=-100].reset_index().drop(columns=['index'])#get rid of clusters with -100 as data is too much
#         table_representation = [] #empty list for table
#         table_representation.append(zip_file)#append table id
#         table_representation.append('')#append page title -> not relevant
#         table_representation.append('')#append wikipedia page id -> not given
#         table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#         table_representation.append('')#append table caption -> not given
#         table_representation.append(df_cleaned['header'].to_list()) #append headers -> not sure if we should do that
#         all_rows_representation = []#representation of all rows
#         for i in range(len(df_cleaned)):
#             row_representation=[] # cell representation of single row
#             row_representation.append([[0,i],[df_cleaned.index[i],str(df_cleaned['name'][i])]])
#             all_rows_representation.append(row_representation) #append single cell representation to representation of all rows
#         table_representation.append(all_rows_representation)#append it to representation of whole table
#         table_representation.append(df_cleaned['cluster_id'].apply(lambda x: [str(x)]).to_list())
#         train_representation.append(table_representation)

/LocalBusiness_goldeniconstruction.com_September2020.csv
/Hotel_hotelderbyzermatt.com_September2020.csv
/LocalBusiness_wepaymorefunding.com_September2020.csv
/LocalBusiness_logo-click.com_September2020.csv
/LocalBusiness_dublinseoexpert.com_September2020.csv
/LocalBusiness_minirepairshops.com_September2020.csv
/Restaurant_foodora.no_September2020.csv
/LocalBusiness_villadonvito.com_September2020.csv
/LocalBusiness_carwashnearme.net.au_September2020.csv
/LocalBusiness_auxiliawebsitedesign.com_September2020.csv
/LocalBusiness_glartent.com_September2020.csv
/LocalBusiness_windscreensnearme.com.au_September2020.csv
/LocalBusiness_calgarycorephysio.ca_September2020.csv
/Hotel_tophotels.ru_September2020.csv
/LocalBusiness_sup-freibad.de_September2020.csv
/Hotel_visitribeesbjerg.es_September2020.csv
/Restaurant_weddingreceptionvenues.com.au_September2020.csv
/Restaurant_lefooding.com_September2020.csv
/LocalBusiness_professionalrugcleaningnearme.com_September2020.csv
/LocalBusiness_currituckc

/LocalBusiness_sportsbookpayperhead.com_September2020.csv
/Hotel_tripadvisor.fr_September2020.csv
/Hotel_esterhotel-krakow.com_September2020.csv
/LocalBusiness_howtorepairholeindrywall.com_September2020.csv
/LocalBusiness_servzip.com_September2020.csv
/Hotel_thai2siam.com_September2020.csv
/LocalBusiness_adamsdirectory.com_September2020.csv
/LocalBusiness_sequimchamber.com_September2020.csv
/LocalBusiness_onlyleaf.com_September2020.csv
/LocalBusiness_golfprostrategies.com_September2020.csv
/LocalBusiness_themagnificentmile.com_September2020.csv
/LocalBusiness_sup-teamevent.de_September2020.csv
/LocalBusiness_starnewsgroup.com_September2020.csv
/LocalBusiness_freedomboatclubfranchise.com_September2020.csv
/LocalBusiness_bresciatourism.it_September2020.csv
/Restaurant_marriott.pt_September2020.csv
/LocalBusiness_vintelli.com_September2020.csv
/LocalBusiness_stocktonnursing.com_September2020.csv
/Hotel_kayak.co.cr_September2020.csv
/LocalBusiness_myhairhelpers.com_September2020.csv
/Resta

/LocalBusiness_companyru.ru_September2020.csv
/LocalBusiness_seattlebootcamps.com_September2020.csv
/Hotel_dailypowwow.com_September2020.csv
/LocalBusiness_valleyviewsnf.com_September2020.csv
/LocalBusiness_affordable-printing.com_September2020.csv
/LocalBusiness_visitdetroitlakes.com_September2020.csv
/LocalBusiness_wdmchamber.org_September2020.csv
/LocalBusiness_04597.com.ua_September2020.csv
/Hotel_hotelbayviewparkmanila.com_September2020.csv
/LocalBusiness_esl-sprachreisen.at_September2020.csv
/LocalBusiness_visitalexandriava.com_September2020.csv
/LocalBusiness_goletachamber.com_September2020.csv
/Hotel_hotelscombined.ie_September2020.csv
/LocalBusiness_foodtronix.com_September2020.csv
/LocalBusiness_liveleague.pl_September2020.csv
/Hotel_marriott.fr_September2020.csv
/LocalBusiness_awareness-coaching-expert.de_September2020.csv
/Hotel_idouanfahotelcasablanca.com_September2020.csv
/LocalBusiness_supseat.com_September2020.csv
/LocalBusiness_visitdenmark.de_September2020.csv
/Hotel_

/LocalBusiness_817area.com_September2020.csv
/Hotel_hotelscombined.com.sg_September2020.csv
/LocalBusiness_brilliantsites.net_September2020.csv
/Restaurant_ellgeebe.com_September2020.csv
/LocalBusiness_indirimhediyeceki.com_September2020.csv
/Hotel_elaurassihotel-algiers.com_September2020.csv
/LocalBusiness_abreumovers.com_September2020.csv
/LocalBusiness_alistweb.org_September2020.csv
/LocalBusiness_monetized.biz_September2020.csv
/LocalBusiness_flood-barriers.com_September2020.csv
/LocalBusiness_sup-teststation.de_September2020.csv
/LocalBusiness_inpatientdrugrehabtexas.net_September2020.csv
/Hotel_romahotellisbon.com_September2020.csv
/LocalBusiness_themeadowshc.com_September2020.csv
/Hotel_everythingperu.com_September2020.csv
/LocalBusiness_onlinezest.org_September2020.csv
/LocalBusiness_eventplanner.be_September2020.csv
/Hotel_gambrinushotelrome.com_September2020.csv
/LocalBusiness_austincityguide.com_September2020.csv
/LocalBusiness_ait-themes.club_September2020.csv
/LocalBusines

/LocalBusiness_mycloudcover.com_September2020.csv
/LocalBusiness_businesscentralmagazine.com_September2020.csv
/LocalBusiness_portsmouthcollaborative.org_September2020.csv
/LocalBusiness_matthewschamber.org_September2020.csv
/Hotel_suha-hotel-apartments-dubai.com_September2020.csv
/LocalBusiness_top40lawfirms.com_September2020.csv
/Hotel_suissehotelgeneva.com_September2020.csv
/Hotel_italianahotelsmilan-rhofair.com_September2020.csv
/Hotel_bestwestern.no_September2020.csv
/LocalBusiness_mckinneychamber.com_September2020.csv
/Hotel_austria-trend.at_September2020.csv
/LocalBusiness_wallrepaircontractor.com_September2020.csv
/LocalBusiness_nephrocare.co.za_September2020.csv
/LocalBusiness_seoshouston.com_September2020.csv
/LocalBusiness_directopedia.us_September2020.csv
/LocalBusiness_schauspielerin-hamburg.de_September2020.csv
/LocalBusiness_bmwpolmaratonpraski.pl_September2020.csv
/LocalBusiness_healthhelpzone.com_September2020.csv
/LocalBusiness_pesacontainercontractors.co.za_September

/LocalBusiness_sezza.org_September2020.csv
/LocalBusiness_homify.sa_September2020.csv
/LocalBusiness_gorospehealth.com_September2020.csv
/Hotel_hotelscombined.dk_September2020.csv
/Hotel_hotellikorpilampiespoo.com_September2020.csv
/LocalBusiness_fwtx.com_September2020.csv
/Hotel_hoteluridinbudapesta.hu_September2020.csv
/LocalBusiness_vialisted.com_September2020.csv
/LocalBusiness_troyohiochamber.com_September2020.csv
/Hotel_themammothmountaininnmammothlakes.com_September2020.csv
/LocalBusiness_parkrapids.com_September2020.csv
/LocalBusiness_uniquewebsitez.com_September2020.csv
/LocalBusiness_0382.ua_September2020.csv
/LocalBusiness_equipmenthirenearme.com.au_September2020.csv
/LocalBusiness_etattata.pl_September2020.csv
/Hotel_thonhotels.com_September2020.csv
/Hotel_botelmarinahotelbratislava.com_September2020.csv
/LocalBusiness_positivesites.net_September2020.csv
/LocalBusiness_awakeningstreatment.com_September2020.csv
/LocalBusiness_tylertexas.com_September2020.csv
/Hotel_urbanihot

/LocalBusiness_freedebtreviews.com_September2020.csv
/Restaurant_visitbroenderslev.dk_September2020.csv
/LocalBusiness_gearstar.com_September2020.csv
/LocalBusiness_dajmudom.pl_September2020.csv
/LocalBusiness_airzonehvac.ca_September2020.csv
/LocalBusiness_arizonaaddiction.com_September2020.csv
/LocalBusiness_flaglerchamber.org_September2020.csv
/LocalBusiness_klonservices.ca_September2020.csv
/LocalBusiness_service-externer-datenschutzbeauftragter.de_September2020.csv
/Restaurant_meltingpot.com_September2020.csv
/LocalBusiness_radontestingcompaniesnearme.com_September2020.csv
/LocalBusiness_11880-gartenbau.com_September2020.csv
/LocalBusiness_millenniumdental.ca_September2020.csv
/Hotel_doriagrandhotelmilan.com_September2020.csv
/LocalBusiness_goportsmouthnh.com_September2020.csv
/Hotel_kayak.ru_September2020.csv
/Restaurant_thebathguide.com_September2020.csv
/LocalBusiness_localplumbernearme.com_September2020.csv
/LocalBusiness_eljottech.pl_September2020.csv
/LocalBusiness_zlot-ewaf

/LocalBusiness_n49.com_September2020.csv
/LocalBusiness_wellness-hotel.info_September2020.csv
/LocalBusiness_evstationslocal.com_September2020.csv
/Hotel_kayak.pt_September2020.csv
/LocalBusiness_tendances-in.com_September2020.csv
/LocalBusiness_sitelistingz.com_September2020.csv
/LocalBusiness_gosonderborg.com_September2020.csv
/LocalBusiness_bowlisting.com_September2020.csv
/LocalBusiness_jeeran.com_September2020.csv
/LocalBusiness_govserv.org_September2020.csv
/LocalBusiness_sup-tourer.de_September2020.csv
/Hotel_skyparkcentralmyeongdonghotel.com_September2020.csv
/Hotel_visitnorway.se_September2020.csv
/LocalBusiness_visitdenmark.dk_September2020.csv
/LocalBusiness_sonomavalley.com_September2020.csv
/LocalBusiness_weebo.us_September2020.csv
/LocalBusiness_happyholidaytime.com_September2020.csv
/Restaurant_zermatt.ch_September2020.csv
/Hotel_floridianhotel-homestead.com_September2020.csv
/Hotel_excelsiorhotellamaddalena.com_September2020.csv
/LocalBusiness_kenzplumbing.com_September

/LocalBusiness_reaach.com_September2020.csv
/LocalBusiness_heb.org_September2020.csv
/LocalBusiness_steinberg-armaturen.de_September2020.csv
/LocalBusiness_leakdetectionplumbingservices.com_September2020.csv
/LocalBusiness_german-news.de_September2020.csv
/LocalBusiness_usamortgagepros.com_September2020.csv
/LocalBusiness_infodirectory.us_September2020.csv
/LocalBusiness_angelinasweb.net_September2020.csv
/LocalBusiness_outpatientla.com_September2020.csv
/LocalBusiness_bizfair.co_September2020.csv
/LocalBusiness_medicalalertforme.com_September2020.csv
/Hotel_positanoarthotelpasitea.com_September2020.csv
/LocalBusiness_casa-smi.de_September2020.csv
/LocalBusiness_bellevuepetadventures.com_September2020.csv
/Hotel_vistana.com_September2020.csv
/LocalBusiness_richlandareachamber.com_September2020.csv
/LocalBusiness_vegasmeansbusiness.com_September2020.csv


In [8]:
# cluster_list=[]

In [13]:
# files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'test tables')) if file.endswith('.csv')]
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_csv(os.path.join(train_test_all_filtered_path,'test tables') + '/{}'.format(zip_file))
#     cluster_list.extend(df['cluster_id'].tolist())

/Hotel_centralplazahotelzurich.com_September2020.csv
/LocalBusiness_prestige-ac.com_September2020.csv
/LocalBusiness_newvisionprojects.com_September2020.csv
/LocalBusiness_medinapaving.com_September2020.csv
/LocalBusiness_beehiveplumbing.com_September2020.csv
/Hotel_swiss-belhotel.com_September2020.csv
/LocalBusiness_sampadaindia.com_September2020.csv
/LocalBusiness_internetmarketing-media.com_September2020.csv
/LocalBusiness_brucesac.com_September2020.csv
/Hotel_islandsindenmark.com_September2020.csv
/Hotel_hoteldeuniverssaintmalo.com_September2020.csv
/LocalBusiness_lafayettecolorado.com_September2020.csv
/LocalBusiness_expatriatelifestyle.com_September2020.csv
/Restaurant_9forks.com_September2020.csv
/Hotel_lessuiteslabellehotelmontreal.com_September2020.csv
/LocalBusiness_selectwaterproofingusa.com_September2020.csv
/LocalBusiness_inlichtet.org_September2020.csv
/Hotel_kishhotels.org_September2020.csv
/LocalBusiness_topdesignfirms.com_September2020.csv
/Restaurant_visitnordjylland.

/LocalBusiness_vannessmainstreet.org_September2020.csv
/Hotel_marblewatershoteljacksonville.com_September2020.csv
/LocalBusiness_topmillionsites.net_September2020.csv
/LocalBusiness_valenciacity.es_September2020.csv
/LocalBusiness_swietelsky.com.au_September2020.csv
/LocalBusiness_hillpeakpups.com_September2020.csv
/LocalBusiness_fixx.co_September2020.csv
/Hotel_momondo.co.uk_September2020.csv
/LocalBusiness_zawya.com_September2020.csv
/Hotel_visitaalborgconvention.com_September2020.csv
/LocalBusiness_couriersnearme.com.au_September2020.csv
/LocalBusiness_blundellcentre.ca_September2020.csv
/LocalBusiness_esl.se_September2020.csv
/Hotel_agoraswissnighthotel-lausanne.com_September2020.csv
/Restaurant_kuechenreise.com_September2020.csv
/LocalBusiness_campingnearme.com.au_September2020.csv
/Hotel_hotellybidkiev.com_September2020.csv
/LocalBusiness_junkyardangel.ca_September2020.csv
/LocalBusiness_santamaria.com_September2020.csv
/LocalBusiness_promolover.com_September2020.csv
/LocalBusine

/LocalBusiness_101healthybody.com_September2020.csv
/LocalBusiness_s-rcchamber.org_September2020.csv
/LocalBusiness_britishbusinessblog.co.uk_September2020.csv
/Hotel_assambleya-nikitskaya-moscow.com_September2020.csv
/LocalBusiness_integrityroofingllc.com_September2020.csv
/Hotel_hotelcesaraugustazaragoza.com_September2020.csv
/Restaurant_visitnyborg.de_September2020.csv
/LocalBusiness_6qube.com_September2020.csv
/Restaurant_lebey.com_September2020.csv
/Restaurant_opentable.it_September2020.csv
/Hotel_secretplaces.pt_September2020.csv
/LocalBusiness_auzdirectory.com.au_September2020.csv
/Hotel_medplaya.eus_September2020.csv
/Restaurant_gosonderborg.dk_September2020.csv
/Restaurant_asiatatler.com_September2020.csv
/LocalBusiness_tfa-szczecin.pl_September2020.csv
/LocalBusiness_oktoberfest-tv.com_September2020.csv
/Restaurant_top-ten-travel-list.com_September2020.csv
/LocalBusiness_esl.co.uk_September2020.csv
/LocalBusiness_0566.com.ua_September2020.csv
/Restaurant_visitmariagerfjord.de

/LocalBusiness_wwvchamber.com_September2020.csv
/Restaurant_visitdenmark.dk_September2020.csv
/Hotel_centralhotelljubljana.com_September2020.csv
/LocalBusiness_ifrit.pl_September2020.csv
/LocalBusiness_windycityguide.com_September2020.csv
/Hotel_elcortezhotellasvegas.com_September2020.csv
/Hotel_urlaubinungarn.hu_September2020.csv
/LocalBusiness_solicitorsnearme.net.au_September2020.csv
/Hotel_kayak.com.mx_September2020.csv
/LocalBusiness_smallbizcrm.com_September2020.csv
/LocalBusiness_ames-peterson.com_September2020.csv
/Hotel_hotelboutiquecatedralvalladolid.com_September2020.csv
/Hotel_northstarcontinentalresorttimisoara.com_September2020.csv
/Hotel_medplaya.com_September2020.csv
/Hotel_dayuse.com_September2020.csv
/LocalBusiness_bubbahead.com_September2020.csv
/Hotel_hoteldelapaixlucerne.com_September2020.csv
/LocalBusiness_619area.com_September2020.csv
/LocalBusiness_xemmau.net_September2020.csv
/Hotel_haervej.com_September2020.csv
/Hotel_visitfyn.dk_September2020.csv
/LocalBusine

/LocalBusiness_delibru.com_September2020.csv
/LocalBusiness_eastcountychamber.org_September2020.csv
/LocalBusiness_clarklawgroupny.com_September2020.csv
/LocalBusiness_midwaychamber.com_September2020.csv
/LocalBusiness_sup-regnitz.de_September2020.csv
/Hotel_royalwestindies-providenciales.com_September2020.csv
/Hotel_h-rzn.com_September2020.csv
/LocalBusiness_hopkinschamber.com_September2020.csv
/Restaurant_vosgesemoi.fr_September2020.csv
/LocalBusiness_getcardable.com_September2020.csv
/LocalBusiness_11880-gebaeudereinigung.com_September2020.csv
/Hotel_momondo.dk_September2020.csv
/Restaurant_tajhotels.com_September2020.csv
/Hotel_kabukiahotelsanfrancisco.com_September2020.csv
/Restaurant_resto.be_September2020.csv
/LocalBusiness_theecostrip.com_September2020.csv
/LocalBusiness_powerbusinesslistings.net_September2020.csv
/LocalBusiness_googalseek.com_September2020.csv
/LocalBusiness_sup-rhein.de_September2020.csv
/LocalBusiness_giaypheplaodong.net.vn_September2020.csv
/Hotel_visitaero

/LocalBusiness_edmontonchamber.com_September2020.csv
/LocalBusiness_czechcrystal.com_September2020.csv
/LocalBusiness_steamaticct.com_September2020.csv
/Restaurant_visitnorway.ru_September2020.csv
/LocalBusiness_bizoftheyear.org_September2020.csv
/LocalBusiness_oceanhillsrecovery.com_September2020.csv
/LocalBusiness_muenchen-datenschutzbeauftragter.com_September2020.csv
/LocalBusiness_twojamuza.pl_September2020.csv
/Hotel_momondo.ie_September2020.csv
/Hotel_schwarzbrunn.at_September2020.csv
/LocalBusiness_commercialroofingcontractorsnearme.com_September2020.csv
/LocalBusiness_best-debt-consolidation-offers.com_September2020.csv
/Hotel_hutchgo.com.cn_September2020.csv
/Hotel_iranhrs.ir_September2020.csv
/LocalBusiness_editorspick.biz_September2020.csv
/Hotel_hotellevipanorama.com_September2020.csv
/LocalBusiness_listurwebsites.com_September2020.csv
/LocalBusiness_11880-tischler.com_September2020.csv
/LocalBusiness_dnasolutions.com.au_September2020.csv
/Hotel_dayuse.nl_September2020.csv


/LocalBusiness_businesseventsperth.com_September2020.csv
/LocalBusiness_orlandojewelrybuyers.com_September2020.csv
/Restaurant_faragoontheroof.com_September2020.csv
/LocalBusiness_visitnorway.ru_September2020.csv
/Restaurant_visitnorway.se_September2020.csv
/LocalBusiness_visitdubai.com_September2020.csv
/LocalBusiness_medinacom.com_September2020.csv
/Restaurant_visitribeesbjerg.cn_September2020.csv
/Restaurant_lonelyplanet.com_September2020.csv
/LocalBusiness_sup-lizenz.de_September2020.csv
/LocalBusiness_xemmau.vn_September2020.csv
/LocalBusiness_convention.at_September2020.csv
/Hotel_hotelcontinentalinnfozdoiguacu.com_September2020.csv
/LocalBusiness_iqsdirectory.com_September2020.csv
/LocalBusiness_bmhba.com_September2020.csv
/LocalBusiness_agfg.com.au_September2020.csv
/LocalBusiness_housepaintingcompaniesnearme.com_September2020.csv
/LocalBusiness_west17avedental.ca_September2020.csv
/LocalBusiness_housecleanersnearme.com.au_September2020.csv
/LocalBusiness_pickeringtonchamber.co

/LocalBusiness_socalreveal.com_September2020.csv
/LocalBusiness_visitvesterhavet.com_September2020.csv
/LocalBusiness_11880-umzug.com_September2020.csv
/LocalBusiness_lethbridgecabs.ca_September2020.csv
/Hotel_genevahotelmiamibeach.com_September2020.csv
/LocalBusiness_logancountychamber.com_September2020.csv
/Hotel_bukkiszallodakbukihotelek.hu_September2020.csv
/Hotel_altiusboutiquehotelnicosia.com_September2020.csv
/LocalBusiness_westgpdental.com_September2020.csv
/LocalBusiness_apsrisk.net_September2020.csv
/LocalBusiness_lalanka.pl_September2020.csv
/LocalBusiness_cusmile.ca_September2020.csv
/LocalBusiness_visit-nordvestkysten.com_September2020.csv
/LocalBusiness_buffalochamber.org_September2020.csv
/Hotel_pro-agent.kz_September2020.csv
/LocalBusiness_visitribeesbjerg.com_September2020.csv
/LocalBusiness_sterlinghospitalmattress.com_September2020.csv
/Hotel_kingshotelbudapest.com_September2020.csv
/Hotel_pirineoshotelfigueres.com_September2020.csv
/Restaurant_viamichelin.at_Septemb

In [24]:
# unique_clusters = np.unique(cluster_list)

In [28]:
# unique_clusters = np.delete(unique_clusters, 0)

In [30]:
# len(unique_clusters)

5923

In [33]:
# pd.DataFrame(unique_clusters.astype(str)).to_csv(path_or_buf=os.path.join(turl_input_path, 'type_vocab_clusters_lb.txt'),sep='\t', index=True, header=False)

In [ ]:
#save list of cluster ids to txt file
# pd.DataFrame(ids).to_csv(path_or_buf=os.path.join(turl_input_path, 'type_vocab_clusters.txt'),sep='\t', index=True, header=False)

# Save as json

In [70]:
len(train_representation)

732

In [5]:
#cannot read numpy integers in json -> encoder for saving
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [20]:
# with open(os.path.join(turl_input_path, 'test_representation_rewritten_cleaned_lb.json'), 'w') as f:
#     json.dump(json.dumps(train_representation, cls=NpEncoder), f)

In [71]:
with open(os.path.join(turl_input_path, 'val_cleaned_representation_rewritten_cleaned_lb.json'), 'w') as f:
    json.dump(json.dumps(train_representation, cls=NpEncoder), f)

In [61]:
train_representation

[['LocalBusiness_prestige-ac.com_September2020.csv',
  '',
  '',
  '',
  '',
  ['Product',
   'Product',
   'Product',
   'Product',
   'Product',
   'Product',
   'Product',
   'Product'],
  [[[[0, 0], [2, 'Prestige Heating and Air Conditioning, LLC']]],
   [[[0, 1], [4, 'Satterlee Plumbing & HVAC']]],
   [[[0, 2], [5, "Snake 'n' Rooter Plumbing Company"]]],
   [[[0, 3], [8, 'Expert Air A/C and Heating']]],
   [[[0, 4], [9, 'Florida Pool Heating']]],
   [[[0, 5], [10, 'AirZone HVAC Services']]],
   [[[0, 6], [11, 'AC Plus Heating & Air']]],
   [[[0, 7], [13, 'Solar Unlimited']]]],
  [['3992'],
   ['2063'],
   ['5237'],
   ['6292'],
   ['6157'],
   ['5842'],
   ['5238'],
   ['5841']]],
 ['LocalBusiness_newvisionprojects.com_September2020.csv',
  '',
  '',
  '',
  '',
  ['Product', 'Product', 'Product', 'Product', 'Product'],
  [[[[0, 0], [5, 'Jim Boles Custom Homes, LLC']]],
   [[[0, 1], [7, 'Brooner Construction & Crane']]],
   [[[0, 2], [9, 'Modern Kitchens & Baths']]],
   [[[0, 3], 